### **Algorithm for Hard-Margin SVM** 💻

Hard-margin SVM can be solved as a quadratic programming (QP) problem. The standard approach is to solve the dual problem, which is often easier to handle. Here are the steps:

#### **Step 1: Set up the Optimization Problem**
The dual optimization problem for a hard-margin SVM is:

**Maximize:**
$$L_D(\mathbf{\alpha}) = \sum_{i=1}^{n} \alpha_i - \frac{1}{2}\sum_{i=1}^{n}\sum_{j=1}^{n} \alpha_i \alpha_j y_i y_j (\mathbf{x}_i \cdot \mathbf{x}_j)$$

**Subject to the constraints:**
$$\sum_{i=1}^{n} \alpha_i y_i = 0$$
$$\alpha_i \ge 0 \quad \text{for } i = 1, \dots, n$$

Here, $n$ is the number of training points, $\alpha_i$ are the Lagrange multipliers, $y_i$ are the class labels (+1 or -1), and $\mathbf{x}_i$ are the feature vectors. The term $(\mathbf{x}_i \cdot \mathbf{x}_j)$ is the **linear kernel**.

#### **Step 2: Implement the Kernel Matrix**
Create a function to compute the kernel matrix **K**, where each element $K_{ij}$ is the dot product of the training samples $\mathbf{x}_i$ and $\mathbf{x}_j$.

* **Function Signature:** `kernel_matrix(X)`
* **Input:** `X`, a matrix of shape `(n_samples, n_features)`.
* **Calculation:** For a linear kernel, the matrix is simply `K = X @ X.T` (matrix multiplication). This is the most computationally efficient way to calculate all dot products.

#### **Step 3: Solve the QP Problem**
You need a QP solver to find the optimal values for $\mathbf{\alpha}$. Since you're implementing from scratch without a dedicated QP library, you'll need to use an iterative optimization algorithm like **Sequential Minimal Optimization (SMO)**. SMO is a popular and efficient algorithm specifically designed for training SVMs.

Here's a high-level overview of the **SMO algorithm**:

1.  **Initialize:** Start with all $\alpha_i = 0$.
2.  **Iterate:**
    * Find two Lagrange multipliers, $\alpha_i$ and $\alpha_j$, to optimize at each step. The selection heuristic is crucial for performance. Typically, you'll choose one $\alpha_i$ that violates the Karush-Kuhn-Tucker (KKT) conditions and a second $\alpha_j$ to maximize the step size.
    * Update $\alpha_i$ and $\alpha_j$ to satisfy the constraints while minimizing the objective function.
    * Clip the new values of $\alpha_i$ and $\alpha_j$ to their allowed range (in hard-margin SVM, this is $[0, \infty)$).
3.  **Repeat:** Continue iterating until the changes in $\alpha$ are very small, indicating convergence.

#### **Step 4: Compute the Weight Vector and Bias**
Once the optimal $\mathbf{\alpha}$ values are found, you can calculate the weight vector $\mathbf{w}$ and the bias $b$.

* **Calculate $\mathbf{w}$:**
    The weight vector is a linear combination of the support vectors.
    $$\mathbf{w} = \sum_{i=1}^{n} \alpha_i y_i \mathbf{x}_i$$
    Note: In practice, you only need to sum over the **support vectors**, which are the data points where $\alpha_i > 0$.

* **Calculate $b$:**
    The bias can be calculated by picking any support vector $(\mathbf{x}_k, y_k)$ and solving the decision boundary equation.
    $$b = y_k - \mathbf{w} \cdot \mathbf{x}_k$$
    A more robust approach is to average the bias over all support vectors to improve numerical stability.

#### **Step 5: Make Predictions**
With $\mathbf{w}$ and $b$, you can now predict the class of a new sample $\mathbf{x}_{new}$.

* **Prediction Function:** `predict(x_new, w, b)`
* **Calculation:**
    $$f(\mathbf{x}_{new}) = \mathbf{w} \cdot \mathbf{x}_{new} + b$$
* **Result:** The predicted class is the sign of $f(\mathbf{x}_{new})$.
    $$\text{prediction} = \text{sign}(f(\mathbf{x}_{new}))$$